In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 0. まずは独自で何も見ないでスコアを出してみる。

In [ ]:
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
train

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

ランダム補完でCabinの欠損値を埋める。この時、ランダムに選択した一つの値で補完している状況に注意し、for文を用いて一つずつ埋めていく。

In [ ]:
train['Cabin'].value_counts()

In [ ]:
import random, math

train_Cabin, test_Cabin = train['Cabin'].values, test['Cabin'].values
unique_CabinDeck = ['B', 'F', 'A', 'G', 'E', 'D', 'C', 'T']
unique_CabinSide = ['P', 'S']

train_Cabin_new = []
for train_c in train_Cabin:
    randCabinDeck = random.choice(unique_CabinDeck)
    randCabinNum = random.randint(0,1894)
    randCabinSide = random.choice(unique_CabinSide)
    if pd.isnull(train_c):
        train_c_new = str(randCabinDeck) + '/' + str(randCabinNum) + '/' + str(randCabinSide)
        train_Cabin_new.append(train_c_new)
    else:
        train_c_new = train_c
        train_Cabin_new.append(train_c_new)

test_Cabin_new = []
for test_c in test_Cabin:
    randCabinDeck = random.choice(unique_CabinDeck)
    randCabinNum = random.randint(0,1894)
    randCabinSide = random.choice(unique_CabinSide)
    if pd.isnull(test_c):
        test_c_new = str(randCabinDeck) + '/' + str(randCabinNum) + '/' + str(randCabinSide)
        test_Cabin_new.append(test_c_new)
    else:
        test_c_new = test_c
        test_Cabin_new.append(test_c_new)

train['Cabin_new'], test['Cabin_new'] = train_Cabin_new, test_Cabin_new
del train['Cabin'], test['Cabin']



In [ ]:
train['Cabin_new'].value_counts()

In [ ]:
test['Cabin_new'].value_counts()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

Transportedされた確率が、/及び_で区切られた数字、文字列ごとに違ってそうに見えるので、CabinとPassengerIDを分けておく

In [ ]:
PIds_train,PIds_test = train['PassengerId'].values, test['PassengerId'].values
Cabins_train, Cabins_test= train['Cabin_new'].values, test['Cabin_new'].values

group_nums_train  = []
num_within_groups_train = []
cabin_decks_train = []
cabin_nums_train = []
cabin_sides_train = []
group_nums_test  = []
num_within_groups_test = []
cabin_decks_test = []
cabin_nums_test = []
cabin_sides_test = []

for PId_train in PIds_train:
    group_num_train,num_within_group_train = PId_train.split('_')
    group_nums_train.append(group_num_train)
    num_within_groups_train.append(num_within_group_train)

for Cabin_train in Cabins_train:
    cabin_deck_train,cabin_num_train,cabin_side_train = Cabin_train.split('/')
    cabin_decks_train.append(cabin_deck_train)
    cabin_nums_train.append(cabin_num_train)
    cabin_sides_train.append(cabin_side_train)
    
for PId_test in PIds_test:
    group_num_test,num_within_group_test = PId_test.split('_')
    group_nums_test.append(group_num_test)
    num_within_groups_test.append(num_within_group_test)

for Cabin_test in Cabins_test:
    cabin_deck_test,cabin_num_test,cabin_side_test = Cabin_test.split('/')
    cabin_decks_test.append(cabin_deck_test)
    cabin_nums_test.append(cabin_num_test)
    cabin_sides_test.append(cabin_side_test)
    
train['GroupNum'] = group_nums_train
train['NumWithinGroup'] = num_within_groups_train
train['CabinDeck'] = cabin_decks_train
train['CabinNum'] = cabin_nums_train
train['CabinSide'] = cabin_sides_train
test['GroupNum'] = group_nums_test
test['NumWithinGroup'] = num_within_groups_test
test['CabinDeck'] = cabin_decks_test
test['CabinNum'] = cabin_nums_test
test['CabinSide'] = cabin_sides_test
del train['Cabin_new'],test['Cabin_new'],

In [ ]:
train.head()

In [ ]:
test.head()

HomePlanetごとのP(Transported=True)を把握しておく

In [ ]:
unique_homeplanet = train['HomePlanet'].unique()
print(unique_homeplanet)

In [ ]:
train_planet_europa = train[train['HomePlanet'] == 'Europa']
train_planet_earth = train[train['HomePlanet'] == 'Earth']
train_planet_mars = train[train['HomePlanet'] == 'Mars']
print(len(train_planet_europa[train_planet_europa['Transported'] == True])/len(train_planet_europa))
print(len(train_planet_earth[train_planet_earth['Transported'] == True])/len(train_planet_earth))
print(len(train_planet_mars[train_planet_mars['Transported'] == True])/len(train_planet_mars))

ランダム補完でHomePlanetの欠損値を埋める

In [ ]:
test['HomePlanet'].value_counts()

In [ ]:
train_homeplanet, test_homeplanet = train['HomePlanet'].values, test['HomePlanet'].values
unique_homeplanet = ['Europa', 'Earth', 'Mars']

train_homeplanet_new = []
for train_h in train_homeplanet:
    randHomePlanet = random.choice(unique_homeplanet)
    if pd.isnull(train_h):
        train_h_new = randHomePlanet
        train_homeplanet_new.append(train_h_new)
    else:
        train_h_new = train_h
        train_homeplanet_new.append(train_h_new)

test_homeplanet_new = []
for test_h in test_homeplanet:
    randHomePlanet = random.choice(unique_homeplanet)
    if pd.isnull(test_h):
        test_h_new = randHomePlanet
        test_homeplanet_new.append(test_h_new)
    else:
        test_h_new = test_h
        test_homeplanet_new.append(test_h_new)

train['HomePlanet_new'],test['HomePlanet_new'] = train_homeplanet_new, test_homeplanet_new
del train['HomePlanet'],test['HomePlanet']
train.head()

In [ ]:
test.head()

In [ ]:
test['HomePlanet_new'].value_counts()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

CryoSleepごとのP(Transported=True)を把握しておく

In [ ]:
train_CryoSleep_T = train[train['CryoSleep'] == True]
train_CryoSleep_F = train[train['CryoSleep'] == False]
print(len(train_CryoSleep_T[train_CryoSleep_T['Transported'] == True])/len(train_CryoSleep_T))
print(len(train_CryoSleep_F[train_CryoSleep_F['Transported'] == True])/len(train_CryoSleep_F))
print(len(train_CryoSleep_T))
print(len(train_CryoSleep_F))

ランダム補完でCryoSleepの欠損値を埋める。

In [ ]:
train['CryoSleep'].value_counts()

In [ ]:
train_CryoSleep, test_CryoSleep = train['CryoSleep'].values, test['CryoSleep'].values
unique_CryoSleep = [True, False]

train_CryoSleep_new = []
for train_cryo in train_CryoSleep:
    randCryoSleep = random.choice(unique_CryoSleep)
    if pd.isnull(train_cryo):
        train_cryo_new = randCryoSleep
        train_CryoSleep_new.append(train_cryo_new)
    else:
        train_cryo_new = train_cryo
        train_CryoSleep_new.append(train_cryo_new)

test_CryoSleep_new = []
for test_cryo in test_CryoSleep:
    randCryoSleep = random.choice(unique_CryoSleep)
    if pd.isnull(test_cryo):
        test_cryo_new = randCryoSleep
        test_CryoSleep_new.append(test_cryo_new)
    else:
        test_cryo_new = test_cryo
        test_CryoSleep_new.append(test_cryo_new)        

train['CryoSleep_new'], test['CryoSleep_new'] = train_CryoSleep_new, test_CryoSleep_new
del train['CryoSleep'], test['CryoSleep']
train.head()

In [ ]:
train['CryoSleep_new'].value_counts()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

DestinationごとのP(Transported=True)を把握しておく

In [ ]:
unique_destinarion = train['Destination'].unique()
print(unique_destinarion)

In [ ]:
train_des1 = train[train['Destination'] == 'TRAPPIST-1e']
train_des2 = train[train['Destination'] == 'PSO J318.5-22']
train_des3 = train[train['Destination'] == '55 Cancri e']
print(len(train_des1[train_des1['Transported'] == True])/len(train_des1))
print(len(train_des2[train_des2['Transported'] == True])/len(train_des2))
print(len(train_des3[train_des3['Transported'] == True])/len(train_des3))
print(len(train_des1),len(train_des2),len(train_des3))

Destinationの欠損値を埋める(最頻値補完)

In [ ]:
train_Des, test_Des = train['Destination'].fillna('TRAPPIST-1e'), test['Destination'].fillna('TRAPPIST-1e')
train['Destination_new'], test['Destination_new'] = train_Des, test_Des
del train['Destination'], test['Destination']
train.head()

In [ ]:
train_des1 = train[train['Destination_new'] == 'TRAPPIST-1e']
train_des2 = train[train['Destination_new'] == 'PSO J318.5-22']
train_des3 = train[train['Destination_new'] == '55 Cancri e']
print(len(train_des1[train_des1['Transported'] == True])/len(train_des1))
print(len(train_des2[train_des2['Transported'] == True])/len(train_des2))
print(len(train_des3[train_des3['Transported'] == True])/len(train_des3))
print(len(train_des1),len(train_des2),len(train_des3))

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

Ageの欠損値を埋める(最頻値補完)

In [ ]:
# from sklearn.ensemble import RandomForestRegressor

# train_Age_predict = train.copy()
# test_Age_predict = test.copy()

# def Age_predictor(train,test):
#     #x = どこにも欠損値がない行すべて
#     #y = どこかには欠損値がある行すべて
#     x_train = train.dropna() 
#     y_train = train[train.isnull().any(axis=1)]
#     x_test = test.dropna()
#     y_test = test[test.isnull().any(axis=1)]
#     #欠損がないデータを使用してランダムフォレストを学習する
#     model_train = RandomForestRegressor(random_state=1).fit(x_train.drop(["Age","Transported"],axis=1), x_train["Age"])
#     model_test = RandomForestRegressor(random_state=1).fit(x_test.drop(["Age","Transported"],axis=1), x_test["Age"])
#     #HomePlanetの欠損値を予測して補完する
#     y_train["Age"] = model_train.predict(y_train.drop(["Age","Transported"],axis=1))
#     y_test["Age"] = model_test.predict(y_test.drop(["Age","Transported"],axis=1))
#     #結合し直す
#     train = pd.concat([x_train,y_train]).sort_index()
#     test = pd.concat([x_test,y_test]).sort_index()
#     return train, test
 
# train_predict, test_predict = Age_predictor(train_Age_predict, test_Age_predict)

In [ ]:
Age_train, Age_test = train['Age'].fillna(train['Age'].median()), test['Age'].fillna(train['Age'].median())
train['Age_new'] = Age_train
test['Age_new'] = Age_test
del train['Age'], test['Age']
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
import matplotlib.pyplot as plt

plt.hist(train['Age_new'].values,bins=80,range=(0,79))

In [ ]:
plt.hist(test['Age_new'].values,bins=80,range=(0,79))

AgelayerごとのP(Transported=True)を把握しておく

In [ ]:
train['Transported'].replace({True: 1, False: 0})
age = train[['Age_new','Transported']].values
age_layer = []
for a in age:
    if a[0] < 12:
        b = '12>'
    elif a[0] >= 12 and a[0] < 25:
        b = '12<'
    elif a[0] >= 25 and a[0] < 40:
        b = '25<'
    elif a[0] >= 40 and a[0] < 55:
        b = '40<'
    elif a[0] >= 55:
        b = '55<'
    age_layer.append([b,a[1]])
age_layer_df = pd.DataFrame(age_layer, columns = ['Age_layer','Transported_rate'])
age_layer_df.groupby('Age_layer').mean()

Ageを12歳未満、以上で分ける

In [ ]:
ages_train, ages_test = train['Age_new'].values, test['Age_new'].values
Under12orNot_train = []
Under12orNot_test = []

for a_train in ages_train:
    if a_train < 12:
        b_train = 1
    else:
        b_train = 0
    Under12orNot_train.append(b_train)

for a_test in ages_test:
    if a_test < 12:
        b_test = 1
    else:
        b_test = 0
    Under12orNot_test.append(b_test)

train['Under12orNot'], test['Under12orNot'] = Under12orNot_train, Under12orNot_test
del train['Age_new'], test['Age_new']
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

VIPごとのP(Transported=True)を把握しておく

In [ ]:
train_VIP_T = train[train['VIP'] == True]
train_VIP_F = train[train['VIP'] == False]
print(len(train_VIP_T[train_VIP_T['Transported'] == True])/len(train_VIP_T))
print(len(train_VIP_F[train_VIP_F['Transported'] == True])/len(train_VIP_F))
print(len(train_VIP_T))
print(len(train_VIP_F))

VIPの欠損値を埋める(最頻値補完)

In [ ]:
train_Vip, test_Vip = train['VIP'].fillna(False), test['VIP'].fillna(False)
train['VIP_new'], test['VIP_new'] = train_Vip, test_Vip
del train['VIP'], test['VIP']
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

各サービスの利用料によるP(Transported=True)を把握しておく

In [ ]:
train_RS_zero = train[train['RoomService'] == 0]
train_RS_notzero = train[train['RoomService'] != 0]
print(len(train_RS_zero[train_RS_zero['Transported'] == True])/len(train_RS_zero))
print(len(train_RS_notzero[train_RS_notzero['Transported'] == True])/len(train_RS_notzero))
print(len(train_RS_notzero),len(train_RS_zero))

In [ ]:
train_FC_zero = train[train['FoodCourt'] == 0]
train_FC_notzero = train[train['FoodCourt'] != 0]
print(len(train_FC_zero[train_FC_zero['Transported'] == True])/len(train_FC_zero))
print(len(train_FC_notzero[train_FC_notzero['Transported'] == True])/len(train_FC_notzero))
print(len(train_FC_notzero),len(train_FC_zero))

In [ ]:
train_SM_zero = train[train['ShoppingMall'] == 0]
train_SM_notzero = train[train['ShoppingMall'] != 0]
print(len(train_SM_zero[train_SM_zero['Transported'] == True])/len(train_SM_zero))
print(len(train_SM_notzero[train_SM_notzero['Transported'] == True])/len(train_SM_notzero))
print(len(train_SM_notzero),len(train_SM_zero))

In [ ]:
train_Spa_zero = train[train['Spa'] == 0]
train_Spa_notzero = train[train['Spa'] != 0]
print(len(train_Spa_zero[train_Spa_zero['Transported'] == True])/len(train_Spa_zero))
print(len(train_Spa_notzero[train_Spa_notzero['Transported'] == True])/len(train_Spa_notzero))
print(len(train_Spa_notzero),len(train_Spa_zero))

In [ ]:
train_VR_zero = train[train['VRDeck'] == 0]
train_VR_notzero = train[train['VRDeck'] != 0]
print(len(train_VR_zero[train_VR_zero['Transported'] == True])/len(train_VR_zero))
print(len(train_VR_notzero[train_VR_notzero['Transported'] == True])/len(train_VR_notzero))
print(len(train_VR_notzero),len(train_VR_zero))

サービスを利用した人の方が別次元に送られていない傾向にある。これはどのサービス内容であっても同じ。
よって、サービスを利用した額のデータの欠損値は全て0で埋め、それを後で足した列を作成する。
さらにその列を額によって何層かに振り分けることで欠損値を解消する。

In [ ]:
RSs_train, RSs_test = train['RoomService'].fillna(0).values, test['RoomService'].fillna(0).values
FCs_train, FCs_test = train['FoodCourt'].fillna(0).values, test['FoodCourt'].fillna(0).values
SMs_train, SMs_test = train['ShoppingMall'].fillna(0).values, test['ShoppingMall'].fillna(0).values
SPs_train, SPs_test = train['Spa'].fillna(0).values, test['Spa'].fillna(0).values
VRs_train, VRs_test = train['VRDeck'].fillna(0).values, test['VRDeck'].fillna(0).values
PaidService_train = RSs_train + FCs_train + SMs_train + SPs_train + VRs_train
PaidService_test = RSs_test + FCs_test + SMs_test + SPs_test + VRs_test

train['PaidService'], test['PaidService'] = PaidService_train, PaidService_test
del train['RoomService'],train['FoodCourt'],train['ShoppingMall'],train['Spa'],train['VRDeck'],test['RoomService'],test['FoodCourt'],test['ShoppingMall'],test['Spa'],test['VRDeck']

plt.hist(PaidService_train,bins=40,range=(0,20000))
plt.hist(PaidService_test,bins=40,range=(0,20000))

In [ ]:
train

In [ ]:
test

In [ ]:
PS_TP = train[['PaidService','Transported']].values
PS_layer = []
zero = 0
first = 0
second = 0
third = 0
for p in PS_TP:
    if p[0] == 0:
        q = 0
        zero = zero + 1
    elif p[0] > 0 and p[0] < 1000:
        q = 1
        first = first + 1
    elif p[0] >= 1000 and p[0] < 2500:
        q = 2
        second = second + 1
    elif p[0] >= 2500:
        q = 3
        third = third + 1
    if p[1] == True:
        r = 1
    elif p[1] == False:
        r = 0
    PS_layer.append([q,r])
PS_layer_df = pd.DataFrame(PS_layer, columns = ['PaidService','Transported_rate'])
PS_layer_df.groupby('PaidService').mean()
# print(zero, first, second, third)

PaidServiceは使った額よりも使ったか使ってないかが重要そう。

In [ ]:
PSs_train, PSs_test = train['PaidService'].values, test['PaidService'].values
new_PSs_train = []
new_PSs_test = []

for PS_train in PSs_train:
    if PS_train == 0:
        new_PS_train = 1
    else:
        new_PS_train = 0
    new_PSs_train.append(new_PS_train)

for PS_test in PSs_test:
    if PS_test == 0:
        new_PS_test = 1
    else:
        new_PS_test = 0
    new_PSs_test.append(new_PS_test)
    
train['No_PaidService'], test['No_PaidService'] = new_PSs_train, new_PSs_test 
del train['PaidService'], test['PaidService']
train.head()


In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

名前の処理。宇宙で起こることだしよくわからないので、念のためイニシャル別にP(Transported=True)を調べてみたり。

In [ ]:
train_Name = train['Name'].fillna('Steaven').values
Initial = []
for n in train_Name:
    Initial.append(n[0])
    
train['Initial'] = Initial
del train['Name']

In [ ]:
I = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','Y','Z']
for i in I:
    train_Ini = train[train['Initial'] == '{}'.format(i)]
    print('イニシャル:{}→'.format(i) + 'P(Transported=True):' + str(round(len(train_Ini[train_Ini['Transported'] == True])/len(train_Ini),2)) + '母数:' + str(len(train_Ini)))


全体のP(Transported=True)は0.5で、母数が多いほど大体0.5に収束しているように見える。
そのため名前はさほど関係ない。
よって名前の列は削除。

In [ ]:
del train['Initial'], test['Name']
train

In [ ]:
test

CabinDeck,CabinNum,CabinSideを裁く。まずはそれぞれのP(Transported=True)を把握する。

In [ ]:
unique_Deck = train['CabinDeck'].unique()
for d in unique_Deck:
    train_CabinDeck = train[train['CabinDeck'] == '{}'.format(d)]
    print('CabinDeck:{}→'.format(d) + 'P(Transported=True):' + str(round(len(train_CabinDeck[train_CabinDeck['Transported'] == True])/len(train_CabinDeck),2)) + '母数:' + str(len(train_CabinDeck)))

In [ ]:
train['CabinNum'] = train['CabinNum'].astype('int64')
train['Transported'].replace({True: 1, False: 0})
CabinNum = train[['CabinNum','Transported']].values
CabinNum_layer = []
for c in CabinNum:
    if c[0] < 100:
        b = 100
    elif c[0] >= 100 and c[0] < 200:
        b = 200
    elif c[0] >= 200 and c[0] < 300:
        b = 300
    elif c[0] >= 300 and c[0] < 400:
        b = 400
    elif c[0] >= 400 and c[0] < 500:
        b = 500
    elif c[0] >= 500 and c[0] < 600:
        b = 600
    elif c[0] >= 600 and c[0] < 700:
        b = 700
    elif c[0] >= 700 and c[0] < 800:
        b = 800
    elif c[0] >= 800 and c[0] < 900:
        b = 900
    elif c[0] >= 900 and c[0] < 1000:
        b = 1000
    elif c[0] >= 1000 and c[0] < 1100:
        b = 1100
    elif c[0] >= 1100 and c[0] < 1200:
        b = 1200
    elif c[0] >= 1200 and c[0] < 1300:
        b = 1300
    elif c[0] >= 1300 and c[0] < 1400:
        b = 1400
    elif c[0] >= 1400 and c[0] < 1500:
        b = 1500
    elif c[0] >= 1500 and c[0] < 1600:
        b = 1600
    elif c[0] >= 1600 and c[0] < 1700:
        b = 1700
    elif c[0] >= 1700:
        b = 1800
    CabinNum_layer.append([b,c[1]])
CabinNum_layer_df = pd.DataFrame(CabinNum_layer, columns = ['CabinNum_layer','Transported_rate'])
CabinNum_layer_df.groupby('CabinNum_layer').mean()

In [ ]:
CabinNum_layer_df.groupby('CabinNum_layer').mean().plot()

Cabinの場所に関係ありそうな分布。

In [ ]:
train_CabinSide_P = train[train['CabinSide'] == 'P']
train_CabinSide_S = train[train['CabinSide'] == 'S']
print(len(train_CabinSide_P[train_CabinSide_P['Transported'] == True])/len(train_CabinSide_P))
print(len(train_CabinSide_S[train_CabinSide_S['Transported'] == True])/len(train_CabinSide_S))
print(len(train_CabinSide_P),len(train_CabinSide_S))

CabinDeckはBC(高),DEF(低),GA(中)に分ける。TはDEF群に入れる。


In [ ]:
Decks_train, Decks_test = train['CabinDeck'].values, test['CabinDeck'].values
CabinDeck_new_train = []
CabinDeck_new_test = []

for Deck_train in Decks_train:
    if Deck_train == 'D' or Deck_train == 'E' or Deck_train == 'F' or Deck_train == 'T':
        Deck_layer_train = 0
    elif Deck_train == 'G' or Deck_train == 'A':
        Deck_layer_train = 1
    elif Deck_train == 'B' or Deck_train == 'C':
        Deck_layer_train = 2
    CabinDeck_new_train.append(Deck_layer_train)

for Deck_test in Decks_test:
    if Deck_test == 'D' or Deck_test == 'E' or Deck_test == 'F' or Deck_test == 'T':
        Deck_layer_test = 0
    elif Deck_test == 'G' or Deck_test == 'A':
        Deck_layer_test = 1
    elif Deck_test == 'B' or Deck_test == 'C':
        Deck_layer_test = 2
    CabinDeck_new_test.append(Deck_layer_test)
    
train['CabinDeck_new'], test['CabinDeck_new'] = CabinDeck_new_train, CabinDeck_new_test
del train['CabinDeck'], test['CabinDeck']
train.head()

In [ ]:
test.head()

CabinNumも低中高に分ける。
1,2,3は真ん中
4,5,6は低い
8,9,10,11は高いので間の7は真ん中。
13,14,15,16,17は低いので間の12は真ん中。
18は真ん中。という波を意識した分類を行う。


In [ ]:
train

In [ ]:
CabinNums_train, CabinNums_test = train['CabinNum'].values, test['CabinNum'].values
CabinNum_new_train = []
CabinNum_new_test = []

for n_train in CabinNums_train:
    if n_train < 300:
        CabinNum_layer_train = 1
    elif n_train >= 300 and n_train < 600:
        CabinNum_layer_train = 0
    elif n_train >= 600 and n_train < 700:
        CabinNum_layer_train = 1
    elif n_train >= 700 and n_train < 1100:
        CabinNum_layer_train = 2
    elif n_train >= 1100 and n_train < 1200:
        CabinNum_layer_train = 1
    elif n_train >= 1200 and n_train < 1700:
        CabinNum_layer_train = 0
    elif n_train >= 1700:
        CabinNum_layer_train = 1
    CabinNum_new_train.append(CabinNum_layer_train)

for n_test in CabinNums_test:
    n_test = int(n_test)
    if n_test < 300:
        CabinNum_layer_test = 1
    elif n_test >= 300 and n_test < 600:
        CabinNum_layer_test = 0
    elif n_test >= 600 and n_test < 700:
        CabinNum_layer_test = 1
    elif n_test >= 700 and n_test < 1100:
        CabinNum_layer_test = 2
    elif n_test >= 1100 and n_test < 1200:
        CabinNum_layer_test = 1
    elif n_test >= 1200 and n_test < 1700:
        CabinNum_layer_test = 0
    elif n_test >= 1700:
        CabinNum_layer_test = 1
    CabinNum_new_test.append(CabinNum_layer_test)
    
train['CabinNum_new'], test['CabinNum_new'] = CabinNum_new_train, CabinNum_new_test
del train['CabinNum'], test['CabinNum']
train.head()

In [ ]:
test.head()

最後にPassengerIDの情報からFamilyNumを取得し、それぞれのTransportedの確率を把握する。

In [ ]:
FamilySize_train, FamilySize_test = train['GroupNum'].value_counts(), test['GroupNum'].value_counts()
FamilySize_dict_train, FamilySize_dict_test = FamilySize_train.to_dict(), FamilySize_test.to_dict()
FamilySize_new_train, FamilySize_new_test = train['GroupNum'].replace(FamilySize_dict_train), test['GroupNum'].replace(FamilySize_dict_test)
train['FamilySize'], test['FamilySize'] = FamilySize_new_train, FamilySize_new_test
del train['GroupNum'], train['NumWithinGroup'], test['GroupNum'], test['NumWithinGroup']
train.head(40)

In [ ]:
test.head(40)

集計の時に、trainとtestを合わせた全体のものを用いる必要があるが、今回は運よくtrainとtestを分ける際に同じグループの人は離れ離れのデータにならないように配慮されていた。
次回から気を付ける。
**trainデータが集計対象のすべてのデータではないことを頭に入れておく。**

In [ ]:
for f in range(1,9):
    train_FamilySize = train[train['FamilySize'] == f]
    print('FamilySize:{}→'.format(f) + 'P(Transported=True):' + str(round(len(train_FamilySize[train_FamilySize['Transported'] == True])/len(train_FamilySize),2)) + '母数:' + str(len(train_FamilySize)))

説明変数として食わせるためにテーブルデータを数値に直す。

In [ ]:
CabinSide_num_train, CabinSide_num_test = train['CabinSide'].replace({'P':0,'S':1}), test['CabinSide'].replace({'P':0,'S':1})
HomePlanet_num_train, HomePlanet_num_test = train['HomePlanet_new'].replace({'Earth':0,'Mars':1,'Europa':2}), test['HomePlanet_new'].replace({'Earth':0,'Mars':1,'Europa':2})
IsCryoSleep_train, IsCryoSleep_test = train['CryoSleep_new'].replace({False:0,True:1}), test['CryoSleep_new'].replace({False:0,True:1})
Destination_num_train, Destination_num_test = train['Destination_new'].replace({'TRAPPIST-1e':0,'PSO J318.5-22':1,'55 Cancri e':2}) ,test['Destination_new'].replace({'TRAPPIST-1e':0,'PSO J318.5-22':1,'55 Cancri e':2})
IsNotVIP_train, IsNotVIP_test = train['VIP_new'].replace({False:1,True:0}), test['VIP_new'].replace({False:1,True:0})

train['CabinSide_num'], test['CabinSide_num'] = CabinSide_num_train, CabinSide_num_test
train['HomePlanet_num'], test['HomePlanet_num'] = HomePlanet_num_train, HomePlanet_num_test
train['IsCryoSleep'], test['IsCryoSleep'] = IsCryoSleep_train, IsCryoSleep_test
train['Destination_num'], test['Destination_num']  = Destination_num_train, Destination_num_test
train['IsNotVIP'], test['IsNotVIP'] = IsNotVIP_train, IsNotVIP_test

del train['CabinSide'],train['HomePlanet_new'],train['CryoSleep_new'],train['Destination_new'],train['VIP_new'],test['CabinSide'],test['HomePlanet_new'],test['CryoSleep_new'],test['Destination_new'],test['VIP_new']
train.head()

In [ ]:
test.head()

とりあえずRFCにかけてみる。

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

X = train[['Under12orNot','No_PaidService','CabinDeck_new','CabinNum_new','FamilySize','CabinSide_num','HomePlanet_num','IsCryoSleep','Destination_num','IsNotVIP']]
y = train[['Transported']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import train_test_split

# X = train[['Under12orNot','No_PaidService','CabinDeck_new','CabinNum_new','FamilySize','CabinSide_num','HomePlanet_num','IsCryoSleep','Destination_num','IsNotVIP']]
# y = train[['Transported']]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)
# prm = {
#         'n_estimators'      : [10,25,50,75,100,300],
#         'max_depth'         : [5,10,15,20,25,30],
#         'max_features'      : ['sqrt', 'log2', 'auto', None],
#         'random_state'      : [0],
#         'n_jobs'            : [-1],
#         'min_samples_split' : [5,10,20,30],
# }
# model = RandomForestClassifier()
# clf = GridSearchCV(estimator = model, param_grid = prm, scoring = "accuracy")
# clf.fit(X_train, y_train.values.ravel())
 
# print(clf.best_estimator_)

RandomForestClassifier(max_depth=5, max_features=None, min_samples_split=30,
                       n_estimators=10, n_jobs=-1, random_state=0)

In [ ]:
best_RFC = RandomForestClassifier(max_depth=5, max_features=None, min_samples_split=30, n_estimators=10, n_jobs=-1, random_state=0)
best_RFC.fit(X_train, y_train.values.ravel())
print(best_RFC.score(X_test, y_test.values.ravel()))

In [ ]:
test_input = test[['Under12orNot','No_PaidService','CabinDeck_new','CabinNum_new','FamilySize','CabinSide_num','HomePlanet_num','IsCryoSleep','Destination_num','IsNotVIP']]
test_predicted = best_RFC.predict(test_input)
print(test_predicted)

In [ ]:
test_result = test.drop(['Under12orNot','No_PaidService','CabinDeck_new','CabinNum_new','FamilySize','CabinSide_num','HomePlanet_num','IsCryoSleep','Destination_num','IsNotVIP'], axis=1)
test_result['Transported'] = test_predicted
test_result=test_result.set_index('PassengerId')
test_result.to_csv('submit_rfc2.csv')

結果ー＞正解率74.304% 560位/659人
かなり悪い

次、以下の手順でやり直してみる。
1. モデルを作ってベースラインをとる(数値データのみ、NULLなしのみ→k-近傍法を使う)
2. 有用な特徴量を探す(カテゴリ特徴量のOneHotEnc,スケール変換,NULL埋め)
3. 他のモデルを試してみて精度を確認する(logistic,SVM,RFC,決定木,勾配boosting回帰木)


# 1. モデルを作ってベースラインをとる(数値データのみ、NULLなしのみ→k-近傍法を使う)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [ ]:
train.head()

In [ ]:
train.dtypes

In [ ]:
train.isnull().sum()

In [ ]:
test.dtypes

In [ ]:
test.isnull().sum()

In [ ]:
from sklearn.model_selection import train_test_split

train = train[['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Transported']].dropna()

X = train[['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']]
y = train[['Transported']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

knn.score(X_test, y_test)

In [ ]:
test_Age = test['Age'].fillna(np.mean(test['Age'])).values
test_RoomService = test['RoomService'].fillna(np.mean(test['RoomService'])).values
test_FoodCourt = test['FoodCourt'].fillna(np.mean(test['FoodCourt'])).values
test_ShoppingMall = test['ShoppingMall'].fillna(np.mean(test['ShoppingMall'])).values
test_Spa = test['Spa'].fillna(np.mean(test['Spa'])).values
test_VRDeck = test['VRDeck'].fillna(np.mean(test['VRDeck'])).values

X_for_submit = pd.DataFrame(list(zip(test_Age,test_RoomService,test_FoodCourt,test_ShoppingMall,test_Spa,test_VRDeck)), columns = ['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck'])
submit = test[['PassengerId']]
submit['Transported'] = knn.predict(X_for_submit)
submit
submit.to_csv('submit_knn2.csv', index=False)

結果ー＞正解率75.029% 661/788
いろいろ試行錯誤した0章よりもいいって、すごいショック。

# 2. このサブミットの結果をより良くするために、有効な特徴量を探し出してみる(=カテゴリ特徴量のOneHotEnc,スケール変換,欠損値補完)

0章にやったデータに対する考察はそのままに、OneHotEncにget_dummies関数を用いたり、標準化を
StandardScalerを用いたりする。

In [ ]:
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

まずは、ランダム補完でCabinの欠損値を埋める。この時、Transportedされた確率が、/及び_で区切られた数字、文字列ごとに違ってそうに見えるので、CabinとPassengerIDの列は分けておく。

In [ ]:
import random, math

train_Cabin, test_Cabin = train['Cabin'].values, test['Cabin'].values
unique_CabinDeck = ['B', 'F', 'A', 'G', 'E', 'D', 'C', 'T']
unique_CabinSide = ['P', 'S']

train_Cabin_new = []
for train_c in train_Cabin:
    randCabinDeck = random.choice(unique_CabinDeck)
    randCabinNum = random.randint(0,1894)
    randCabinSide = random.choice(unique_CabinSide)
    if pd.isnull(train_c):
        train_c_new = str(randCabinDeck) + '/' + str(randCabinNum) + '/' + str(randCabinSide)
        train_Cabin_new.append(train_c_new)
    else:
        train_c_new = train_c
        train_Cabin_new.append(train_c_new)

test_Cabin_new = []
for test_c in test_Cabin:
    randCabinDeck = random.choice(unique_CabinDeck)
    randCabinNum = random.randint(0,1894)
    randCabinSide = random.choice(unique_CabinSide)
    if pd.isnull(test_c):
        test_c_new = str(randCabinDeck) + '/' + str(randCabinNum) + '/' + str(randCabinSide)
        test_Cabin_new.append(test_c_new)
    else:
        test_c_new = test_c
        test_Cabin_new.append(test_c_new)
        
train['Cabin_new'], test['Cabin_new'] = train_Cabin_new, test_Cabin_new
del train['Cabin'], test['Cabin']


PIds_train,PIds_test = train['PassengerId'].values, test['PassengerId'].values
Cabins_train, Cabins_test= train['Cabin_new'].values, test['Cabin_new'].values

group_nums_train  = []
num_within_groups_train = []
cabin_decks_train = []
cabin_nums_train = []
cabin_sides_train = []
group_nums_test  = []
num_within_groups_test = []
cabin_decks_test = []
cabin_nums_test = []
cabin_sides_test = []

for PId_train in PIds_train:
    group_num_train,num_within_group_train = PId_train.split('_')
    group_nums_train.append(group_num_train)
    num_within_groups_train.append(num_within_group_train)

for Cabin_train in Cabins_train:
    cabin_deck_train,cabin_num_train,cabin_side_train = Cabin_train.split('/')
    cabin_decks_train.append(cabin_deck_train)
    cabin_nums_train.append(cabin_num_train)
    cabin_sides_train.append(cabin_side_train)

for PId_test in PIds_test:
    group_num_test,num_within_group_test = PId_test.split('_')
    group_nums_test.append(group_num_test)
    num_within_groups_test.append(num_within_group_test)

for Cabin_test in Cabins_test:
    cabin_deck_test,cabin_num_test,cabin_side_test = Cabin_test.split('/')
    cabin_decks_test.append(cabin_deck_test)
    cabin_nums_test.append(cabin_num_test)
    cabin_sides_test.append(cabin_side_test)
    
train['GroupNum'] = group_nums_train
train['NumWithinGroup'] = num_within_groups_train
train['CabinDeck'] = cabin_decks_train
train['CabinNum'] = cabin_nums_train
train['CabinSide'] = cabin_sides_train
test['GroupNum'] = group_nums_test
test['NumWithinGroup'] = num_within_groups_test
test['CabinDeck'] = cabin_decks_test
test['CabinNum'] = cabin_nums_test
test['CabinSide'] = cabin_sides_test
del train['Cabin_new'],test['Cabin_new']

CabinDeck,CabinNum,CabinSideそれぞれのP(Transported=True)は、特徴量として若干有効であり、特にCabinNumはP(Transported=True)に対して分布を持っているように見えることが0章でわかった。
また、PassengerIDの情報からFamilyNumを取得し、それぞれのTransportedの確率を把握すると、多い方がRiskが高かったことがわかっている。

In [ ]:
# CabinDeckはBC(High),DEF(Low),GA(Middle)に分ける。TはDEF群に入れる。 これをワンホットエンコードする。
# 追記:そのままにしてLabelEncodingした。
# CabinSideもワンホットエンコードする。
# PassengerIDからFamilyNumを計算する。この数値はカテゴリではないのでワンホットしないでそのまま処理する。

Decks_train, Decks_test = train['CabinDeck'].values, test['CabinDeck'].values
# CabinDeck_new_train = []
# CabinDeck_new_test = []

# for Deck_train in Decks_train:
#     if Deck_train == 'D' or Deck_train == 'E' or Deck_train == 'F' or Deck_train == 'T':
#         Deck_layer_train = 'LowRisk'
#     elif Deck_train == 'G' or Deck_train == 'A':
#         Deck_layer_train = 'MiddleRisk'
#     elif Deck_train == 'B' or Deck_train == 'C':
#         Deck_layer_train = 'HighRisk'
#     CabinDeck_new_train.append(Deck_layer_train)

# for Deck_test in Decks_test:
#     if Deck_test == 'D' or Deck_test == 'E' or Deck_test == 'F' or Deck_test == 'T':
#         Deck_layer_test = 'LowRisk'
#     elif Deck_test == 'G' or Deck_test == 'A':
#         Deck_layer_test = 'MiddleRisk'
#     elif Deck_test == 'B' or Deck_test == 'C':
#         Deck_layer_test = 'HighRisk'
#     CabinDeck_new_test.append(Deck_layer_test)
    
# train['CabinDeck_new'], test['CabinDeck_new'] = CabinDeck_new_train, CabinDeck_new_test
# del train['CabinDeck'], test['CabinDeck']


# train = pd.get_dummies(train, columns=['CabinDeck_new'])
# test = pd.get_dummies(test, columns=['CabinDeck_new'])

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Decks_train)
le.fit(Decks_test)
Decks_test_new = le.transform(Decks_test)
Decks_train_new = le.transform(Decks_train)
train['CabinDeck_new'], test['CabinDeck_new'] = Decks_train_new, Decks_test_new 
del train['CabinDeck'], test['CabinDeck']

CabinSide_S_train, CabinSide_S_test = train['CabinSide'].replace({'P':0,'S':1}), test['CabinSide'].replace({'P':0,'S':1})
train['isCabinSide_S'], test['isCabinSide_S'] = CabinSide_S_train, CabinSide_S_test

FamilySize_train, FamilySize_test = train['GroupNum'].value_counts(), test['GroupNum'].value_counts()
FamilySize_dict_train, FamilySize_dict_test = FamilySize_train.to_dict(), FamilySize_test.to_dict()
FamilySize_new_train, FamilySize_new_test = train['GroupNum'].replace(FamilySize_dict_train), test['GroupNum'].replace(FamilySize_dict_test)
train['FamilySize'], test['FamilySize'] = FamilySize_new_train, FamilySize_new_test
del train['GroupNum'], train['NumWithinGroup'], train['CabinSide'], test['GroupNum'], test['NumWithinGroup'], test['CabinSide']

train

HomePlanet,CryoSleep,Destination,Age,VIPの欠損値をなんとかした後カテゴリ系の特徴量にワンホットをアプライする。
また名前はさほど関係ないことがわかっているので名前の列は削除し、CabinNumがobjectだったのでintにした。

In [ ]:
# HomePlanet:ランダム補完

train_homeplanet, test_homeplanet = train['HomePlanet'].values, test['HomePlanet'].values
unique_homeplanet = ['Europa', 'Earth', 'Mars']

train_homeplanet_new = []
for train_h in train_homeplanet:
    randHomePlanet = random.choice(unique_homeplanet)
    if pd.isnull(train_h):
        train_h_new = randHomePlanet
        train_homeplanet_new.append(train_h_new)
    else:
        train_h_new = train_h
        train_homeplanet_new.append(train_h_new)

test_homeplanet_new = []
for test_h in test_homeplanet:
    randHomePlanet = random.choice(unique_homeplanet)
    if pd.isnull(test_h):
        test_h_new = randHomePlanet
        test_homeplanet_new.append(test_h_new)
    else:
        test_h_new = test_h
        test_homeplanet_new.append(test_h_new)

train['HomePlanet_new'],test['HomePlanet_new'] = train_homeplanet_new, test_homeplanet_new
del train['HomePlanet'],test['HomePlanet']

# CryoSleep:ランダム補完


train_CryoSleep, test_CryoSleep = train['CryoSleep'].values, test['CryoSleep'].values
unique_CryoSleep = [True, False]

train_CryoSleep_new = []
for train_cryo in train_CryoSleep:
    randCryoSleep = random.choice(unique_CryoSleep)
    if pd.isnull(train_cryo):
        train_cryo_new = randCryoSleep
        train_CryoSleep_new.append(train_cryo_new)
    else:
        train_cryo_new = train_cryo
        train_CryoSleep_new.append(train_cryo_new)

test_CryoSleep_new = []
for test_cryo in test_CryoSleep:
    randCryoSleep = random.choice(unique_CryoSleep)
    if pd.isnull(test_cryo):
        test_cryo_new = randCryoSleep
        test_CryoSleep_new.append(test_cryo_new)
    else:
        test_cryo_new = test_cryo
        test_CryoSleep_new.append(test_cryo_new)        

train['CryoSleep_new'], test['CryoSleep_new'] = train_CryoSleep_new, test_CryoSleep_new
del train['CryoSleep'], test['CryoSleep']

# Destination:最頻値補完

train_Des, test_Des = train['Destination'].fillna('TRAPPIST-1e'), test['Destination'].fillna('TRAPPIST-1e')
train['Destination_new'], test['Destination_new'] = train_Des, test_Des
del train['Destination'], test['Destination']

# Age:最頻値補完

Age_train, Age_test = train['Age'].fillna(train['Age'].median()), test['Age'].fillna(train['Age'].median())
train['Age_new'] = Age_train
test['Age_new'] = Age_test
del train['Age'], test['Age']

# VIP:最頻値補完
train_Vip, test_Vip = train['VIP'].fillna(False), test['VIP'].fillna(False)
train['VIP_new'], test['VIP_new'] = train_Vip, test_Vip
del train['VIP'], test['VIP']

#ワンホット
train = pd.get_dummies(train, columns=['HomePlanet_new','Destination_new'])
test = pd.get_dummies(test, columns=['HomePlanet_new','Destination_new'])
IsCryoSleep_train, IsCryoSleep_test = train['CryoSleep_new'].replace({False:0,True:1}), test['CryoSleep_new'].replace({False:0,True:1})
train['IsCryoSleep'], test['IsCryoSleep'] = IsCryoSleep_train, IsCryoSleep_test
IsNotVIP_train, IsNotVIP_test = train['VIP_new'].replace({False:1,True:0}), test['VIP_new'].replace({False:1,True:0})
train['IsNotVIP'], test['IsNotVIP'] = IsNotVIP_train, IsNotVIP_test

del train['CryoSleep_new'],train['VIP_new'],train['Name'],test['CryoSleep_new'],test['VIP_new'],test['Name']
train['CabinNum'], test['CabinNum'] = train['CabinNum'].astype('int64'), test['CabinNum'].astype('int64')


サービスを利用した額のデータの欠損値は全て0で埋め、それを後で足した列を作成することで欠損値を解消する。

In [ ]:
RSs_train, RSs_test = train['RoomService'].fillna(0).values, test['RoomService'].fillna(0).values
FCs_train, FCs_test = train['FoodCourt'].fillna(0).values, test['FoodCourt'].fillna(0).values
SMs_train, SMs_test = train['ShoppingMall'].fillna(0).values, test['ShoppingMall'].fillna(0).values
SPs_train, SPs_test = train['Spa'].fillna(0).values, test['Spa'].fillna(0).values
VRs_train, VRs_test = train['VRDeck'].fillna(0).values, test['VRDeck'].fillna(0).values
PaidService_train = RSs_train + FCs_train + SMs_train + SPs_train + VRs_train
PaidService_test = RSs_test + FCs_test + SMs_test + SPs_test + VRs_test

train['PaidService'], test['PaidService'] = PaidService_train, PaidService_test
del train['RoomService'],train['FoodCourt'],train['ShoppingMall'],train['Spa'],train['VRDeck'],test['RoomService'],test['FoodCourt'],test['ShoppingMall'],test['Spa'],test['VRDeck']

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split


X = train[['CabinNum','CabinDeck_new','isCabinSide_S','FamilySize','Age_new', 'HomePlanet_new_Earth', 'HomePlanet_new_Europa', 'HomePlanet_new_Mars', 'Destination_new_55 Cancri e', 'Destination_new_PSO J318.5-22', 'Destination_new_TRAPPIST-1e', 'IsCryoSleep', 'IsNotVIP', 'PaidService']]
y = train[['Transported']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

knn = KNeighborsClassifier()
knn.fit(X_train_scaled, y_train)
knn.score(X_test_scaled, y_test)

In [ ]:
submit = test[['PassengerId']]
X_for_submit = test[['CabinNum','CabinDeck_new','isCabinSide_S','FamilySize','Age_new', 'HomePlanet_new_Earth', 'HomePlanet_new_Europa', 'HomePlanet_new_Mars', 'Destination_new_55 Cancri e', 'Destination_new_PSO J318.5-22', 'Destination_new_TRAPPIST-1e', 'IsCryoSleep', 'IsNotVIP', 'PaidService']]
X_for_submit_scaled = scaler.transform(X_for_submit)
submit['Transported'] = knn.predict(X_for_submit_scaled)
submit
submit.to_csv('submit_knn3.csv', index=False)

結果：0.71335と過去最低となった。

0章での知見をもう少し追加する。

In [ ]:
# #年齢を12歳以下かどうかで分ける

# ages_train, ages_test = train['Age_new'].values, test['Age_new'].values
# Under12orNot_train = []
# Under12orNot_test = []

# for a_train in ages_train:
#     if a_train < 12:
#         b_train = 1
#     else:
#         b_train = 0
#     Under12orNot_train.append(b_train)

# for a_test in ages_test:
#     if a_test < 12:
#         b_test = 1
#     else:
#         b_test = 0
#     Under12orNot_test.append(b_test)

# train['Under12orNot'], test['Under12orNot'] = Under12orNot_train, Under12orNot_test
# del train['Age_new'], test['Age_new']

# #サービスの利用があるかないかで分ける

# PSs_train, PSs_test = train['PaidService'].values, test['PaidService'].values
# new_PSs_train = []
# new_PSs_test = []

# for PS_train in PSs_train:
#     if PS_train == 0:
#         new_PS_train = 1
#     else:
#         new_PS_train = 0
#     new_PSs_train.append(new_PS_train)

# for PS_test in PSs_test:
#     if PS_test == 0:
#         new_PS_test = 1
#     else:
#         new_PS_test = 0
#     new_PSs_test.append(new_PS_test)
    
# train['No_PaidService'], test['No_PaidService'] = new_PSs_train, new_PSs_test 
# del train['PaidService'], test['PaidService']

In [ ]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import train_test_split


# X_new = train[['CabinNum','CabinDeck_new_HighRisk','CabinDeck_new_LowRisk','CabinDeck_new_MiddleRisk','isCabinSide_S','FamilySize','Under12orNot', 'HomePlanet_new_Earth', 'HomePlanet_new_Europa', 'HomePlanet_new_Mars', 'Destination_new_55 Cancri e', 'Destination_new_PSO J318.5-22', 'Destination_new_TRAPPIST-1e', 'IsCryoSleep', 'IsNotVIP', 'No_PaidService']]

# X_train, X_test, y_train, y_test = train_test_split(X_new, y, random_state=0)

# scaler = StandardScaler()
# scaler.fit(X_train)

# X_train_scaled = scaler.transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# knn = KNeighborsClassifier()
# knn.fit(X_train_scaled, y_train)
# knn.score(X_test_scaled, y_test)

下がったので元に戻す

In [ ]:
train.head()

In [ ]:
test.head()

# 3. 他のモデルを試してみて精度を確認する(logistic,SVM,RFC,決定木,勾配boosting回帰木)

In [ ]:
X = train[['CabinNum','CabinDeck_new','isCabinSide_S','FamilySize','Age_new', 'HomePlanet_new_Earth', 'HomePlanet_new_Europa', 'HomePlanet_new_Mars', 'Destination_new_55 Cancri e', 'Destination_new_PSO J318.5-22', 'Destination_new_TRAPPIST-1e', 'IsCryoSleep', 'IsNotVIP', 'PaidService']]
y = train[['Transported']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

#SVM
from sklearn.svm import LinearSVC
svc = LinearSVC()
svc.fit(X_train_scaled,y_train)

#RFC
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train_scaled,y_train)

#勾配boosting回帰木
from sklearn.ensemble import GradientBoostingClassifier
gbrt = GradientBoostingClassifier()
gbrt.fit(X_train_scaled,y_train)

#ロジスティック回帰
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_scaled,y_train)

print('SVM:' + str(svc.score(X_test_scaled,y_test)))
print('RFC:' + str(rf.score(X_test_scaled,y_test)))
print('勾配Boosting回帰木:' + str(gbrt.score(X_test_scaled,y_test)))
print('ロジスティック回帰:' + str(lr.score(X_test_scaled,y_test)))

勾配Boosting回帰木でチューニングしてみる

In [ ]:
# from sklearn.model_selection import GridSearchCV

# parameters = {
#     "n_estimators":[i for i in range(10,100,10)],
#     "learning_rate":[0.01,0.03,0.05,0.1,0.15,0.2],
#     "max_depth":[3,5,7,8,9,10],
#     "min_samples_split": [2,4,5,10,12,16],
#     "random_state":[3],
# }

# gbrt = GradientBoostingClassifier()
# clf = GridSearchCV(estimator = gbrt, param_grid = parameters, cv=5)
# clf.fit(X_train_scaled, y_train.values.ravel())

# print(clf.best_estimator_)

GradientBoostingClassifier(max_depth=5, min_samples_split=5, n_estimators=50,
                           random_state=3)

In [ ]:
X = train[['CabinNum','CabinDeck_new','isCabinSide_S','FamilySize','Age_new', 'HomePlanet_new_Earth', 'HomePlanet_new_Europa', 'HomePlanet_new_Mars', 'Destination_new_55 Cancri e', 'Destination_new_PSO J318.5-22', 'Destination_new_TRAPPIST-1e', 'IsCryoSleep', 'IsNotVIP', 'PaidService']]
y = train[['Transported']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=0)

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

gbrt_best = GradientBoostingClassifier(max_depth=5, min_samples_split=5, n_estimators=53, random_state=123, learning_rate=0.131)
gbrt_best.fit(X_train_scaled,y_train)
gbrt_best.score(X_test_scaled,y_test)

In [ ]:
X_for_submit = test[['CabinNum','CabinDeck_new','isCabinSide_S','FamilySize','Age_new', 'HomePlanet_new_Earth', 'HomePlanet_new_Europa', 'HomePlanet_new_Mars', 'Destination_new_55 Cancri e', 'Destination_new_PSO J318.5-22', 'Destination_new_TRAPPIST-1e', 'IsCryoSleep', 'IsNotVIP', 'PaidService']]
submit = test[['PassengerId']]

X_for_submit_scaled = scaler.transform(X_for_submit)
submit['Transported'] = gbrt_best.predict(X_for_submit_scaled)
submit.to_csv('submit_gbrt2.csv', index=False)

# 4. 性能が良いと話題のCATboostを使ってみる

In [ ]:
from catboost import Pool
from catboost import CatBoostClassifier

X = train[['CabinNum','CabinDeck_new','isCabinSide_S','FamilySize','Age_new', 'HomePlanet_new_Earth', 'HomePlanet_new_Europa', 'HomePlanet_new_Mars', 'Destination_new_55 Cancri e', 'Destination_new_PSO J318.5-22', 'Destination_new_TRAPPIST-1e', 'IsCryoSleep', 'IsNotVIP', 'PaidService']].values
y = train[['Transported']].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=0)

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

CatBoost_train = Pool(X_train_scaled, label=y_train) 
CatBoost_test = Pool(X_test_scaled, label=y_test)

In [ ]:
# cbc = CatBoostClassifier()
# param_grid = {'depth': [1,2,3],
#               'learning_rate' : [0.001,0.005,0.01,0.05],
#               'l2_leaf_reg': [1,2,3],
#               'iterations': [300]}

# grid_result = GridSearchCV(estimator = cbc,
#                            param_grid = param_grid,
#                            scoring = 'accuracy',
#                            cv = 5,
#                            verbose= False,
#                            return_train_score = False,
#                            n_jobs = -1)

# grid_result.fit(X_train_scaled, y_train.ravel())
# print(grid_result.best_estimator_)
# print(grid_result.best_params_)

{'depth': 4, 'iterations': 300, 'l2_leaf_reg': 1, 'learning_rate': 0.01}

In [ ]:
cbc_best = CatBoostClassifier(depth = 2, iterations= 1000, l2_leaf_reg= 1, learning_rate= 0.001)
cbc_best.fit(X_train_scaled, y_train)
ans=cbc_best.predict(X_test_scaled)
ans_new = []
for a in ans:
    if a == 'True':
        b = True
    else:
        b = False
    ans_new.append(b)

acc = 0
for y, a in zip(y_test, ans_new):
    if y == [a]:
        acc = acc + 1
    else:continue
print(acc/len(y_test))

In [ ]:
submit = test[['PassengerId']]
X_for_submit = test[['CabinNum','CabinDeck_new','isCabinSide_S','FamilySize','Age_new', 'HomePlanet_new_Earth', 'HomePlanet_new_Europa', 'HomePlanet_new_Mars', 'Destination_new_55 Cancri e', 'Destination_new_PSO J318.5-22', 'Destination_new_TRAPPIST-1e', 'IsCryoSleep', 'IsNotVIP', 'PaidService']]
X_for_submit_scaled = scaler.transform(X_for_submit)
submit['Transported'] = cbc_best.predict(X_for_submit_scaled)
submit.to_csv('submit_cbc4.csv', index=False)

# 5. LazyClassifierで試してみる

https://www.kaggle.com/code/yukiyokochi/spaceship-titanic-lazyclassifier

で試した。

In [ ]:
# from IPython.display import clear_output
# !pip3 install -U lazypredict
# !pip3 install -U pandas #Upgrading pandas
# !pip3 install -U scikit-learn==0.24.2

# clear_output()

In [ ]:
# import lazypredict
# from lazypredict.Supervised import LazyClassifier

# X = train[['CabinNum', 'CabinDeck_new', 'isCabinSide_S', 'FamilySize', 'Age_new', 'HomePlanet_new_Earth', 'HomePlanet_new_Europa', 'HomePlanet_new_Mars', 'Destination_new_55 Cancri e', 'Destination_new_PSO J318.5-22', 'Destination_new_TRAPPIST-1e', 'IsCryoSleep', 'IsNotVIP', 'PaidService']]
# y = train[['Transported']]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=0)
# lc = LazyClassifier(verbose=0,
#                     ignore_warnings=True,
#                     custom_metric=None,
#                     predictions=False,
#                     random_state=12,
#                     classifiers='all')

# models, predictions = lc.fit(X_train , X_test , y_train , y_test)
# clear_output()